In [1]:
from operator import neg
import pandas as pd
import pickle
import requests
from requests.structures import CaseInsensitiveDict
import json
import time

In [2]:
client_id = ""
client_secret = ""

In [3]:
# Get a bearer token.
petfinder_token_url = "https://api.petfinder.com/v2/oauth2/token"

data = {'grant_type' : 'client_credentials', 'client_id' : client_id , 'client_secret' : client_secret}

response = requests.post(petfinder_token_url, data = data)

json_response = json.loads(response.content)

bearer_token = json_response['access_token']

In [4]:
# Get Best Friends Society Organization.
petfinder_get_organizations_url = "https://api.petfinder.com/v2/organizations"

headers = CaseInsensitiveDict()
headers["Accept"] = "application/json"
headers["Authorization"] = "Bearer " + bearer_token

params = {'name' : 'Best Friends Animal Society'}

response = requests.get(petfinder_get_organizations_url, params = params, headers = headers)

json_response = json.loads(response.content)

for i in range(5):
    print(json_response['organizations'][i])

{'id': 'UT81', 'name': 'Best Friends Animal Society-Utah', 'email': 'utahadoptions@bestfriends.org', 'phone': '(801) 574-2454', 'address': {'address1': '2005 S 1100 E', 'address2': None, 'city': 'Salt Lake City', 'state': 'UT', 'postcode': '84106', 'country': 'US'}, 'hours': {'monday': '10 am - 7 pm', 'tuesday': '10 am - 7 pm', 'wednesday': '10 am - 7 pm', 'thursday': '10 am - 7 pm', 'friday': '10 am - 7 pm', 'saturday': '10 am - 7 pm', 'sunday': '10 am - 7 pm'}, 'url': 'https://www.petfinder.com/member/us/ut/salt-lake-city/best-friends-animal-society-utah-ut81/?referrer_id=b23ab561-58a7-4b32-b872-5418ff49f309', 'website': 'http://utah.bestfriends.org', 'mission_statement': 'The mission of Best Friends Animal Society is to bring about a time when there are No More Homeless Pets....', 'adoption': {'policy': 'Due to Covid-19, our SLC Lifesaving Center is temporarily closed. All pets are currently in foster homes and adoption meet...', 'url': 'https://utah.bestfriends.org/get-involved/ado

In [5]:
# The Best Friends Animal Society's main office is the one with id "UT77".
# Getting some profiles from that organization.
petfinder_get_animals_url = "https://api.petfinder.com/v2/animals"

headers = CaseInsensitiveDict()
headers["Accept"] = "application/json"
headers["Authorization"] = "Bearer " + bearer_token

organization_id = "UT77"
limit = 100

params = {'organization' : organization_id, 'limit' : limit}

response = requests.get(petfinder_get_animals_url, params = params, headers = headers)

json_response = json.loads(response.content)

print(json_response)

{'animals': [{'id': 58694102, 'organization_id': 'UT77', 'url': 'https://www.petfinder.com/dog/candy-corn-58694102/ut/kanab/best-friends-animal-society-ut77/?referrer_id=b23ab561-58a7-4b32-b872-5418ff49f309', 'type': 'Dog', 'species': 'Dog', 'breeds': {'primary': 'Shepherd', 'secondary': None, 'mixed': False, 'unknown': False}, 'colors': {'primary': 'Yellow / Tan / Blond / Fawn', 'secondary': None, 'tertiary': None}, 'age': 'Adult', 'gender': 'Female', 'size': 'Medium', 'coat': None, 'attributes': {'spayed_neutered': False, 'house_trained': False, 'declawed': None, 'special_needs': False, 'shots_current': True}, 'environment': {'children': None, 'dogs': None, 'cats': None}, 'tags': [], 'name': 'Candy Corn', 'description': 'This is Candy Corn. She is the amazing mother of the Howl-o-ween Litter. She takes amazing care of her 8...', 'organization_animal_id': 'BKNB-A-41755', 'photos': [{'small': 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/58694102/1/?bust=1667001138&width=100', 'med

In [6]:
print(len(json_response['animals']))

100


In [7]:
# Harmonize data.
animals = json_response['animals']
harmonized_animals = []

for animal in animals:

    harmonized_animal = {'Type' : animal['type'], 'Name' : animal['name'], 'Age' : animal['age'],\
                           'Breed1' : animal['breeds']['primary'], 'Breed2' : animal['breeds']['secondary'], \
                           'Gender' : animal['gender'], 'Color1' : animal['colors']['primary'], \
                           'Color2' : animal['colors']['secondary'], 'Color3' : animal['colors']['tertiary'], \
                           'MaturitySize' : animal['size'], 'FurLength' : None, 'Vaccinated' : animal['attributes']['shots_current'], \
                           'Dewormed' : None, 'Sterilized' : animal['attributes']['spayed_neutered'], 'Health' : None, \
                           'Quantity' : 1, 'Fee' : None, 'State' : animal['contact']['address']['state'], 'RescuerID' : None, \
                           'VideoAmt' : len(animal['videos']), 'Description' : animal['description'], 'PetID' : animal['id'], \
                           'PhotoAmt' : len(animal['photos']), 'AdoptionSpeed' : None, 'id' : animal['id'] }
    harmonized_animals.append(harmonized_animal)

print(harmonized_animals)

[{'Type': 'Dog', 'Name': 'Candy Corn', 'Age': 'Adult', 'Breed1': 'Shepherd', 'Breed2': None, 'Gender': 'Female', 'Color1': 'Yellow / Tan / Blond / Fawn', 'Color2': None, 'Color3': None, 'MaturitySize': 'Medium', 'FurLength': None, 'Vaccinated': True, 'Dewormed': None, 'Sterilized': False, 'Health': None, 'Quantity': 1, 'Fee': None, 'State': 'UT', 'RescuerID': None, 'VideoAmt': 0, 'Description': 'This is Candy Corn. She is the amazing mother of the Howl-o-ween Litter. She takes amazing care of her 8...', 'PetID': 58694102, 'PhotoAmt': 2, 'AdoptionSpeed': None, 'id': 58694102}, {'Type': 'Cat', 'Name': 'Ponyo', 'Age': 'Baby', 'Breed1': 'Domestic Short Hair', 'Breed2': None, 'Gender': 'Female', 'Color1': 'Brown / Chocolate', 'Color2': None, 'Color3': None, 'MaturitySize': 'Small', 'FurLength': None, 'Vaccinated': True, 'Dewormed': None, 'Sterilized': True, 'Health': None, 'Quantity': 1, 'Fee': None, 'State': 'UT', 'RescuerID': None, 'VideoAmt': 0, 'Description': None, 'PetID': 58681737, 'P

In [8]:
df = pd.DataFrame(harmonized_animals)

In [9]:
df.head(10)

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,...,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,AdoptionSpeed,id
0,Dog,Candy Corn,Adult,Shepherd,None,Female,Yellow / Tan / Blond / Fawn,None,None,Medium,...,1,None,UT,None,0,This is Candy Corn. She is the amazing mother ...,58694102,2,None,58694102
1,Cat,Ponyo,Baby,Domestic Short Hair,None,Female,Brown / Chocolate,None,None,Small,...,1,None,UT,None,0,None,58681737,1,None,58681737
2,Cat,Petrel,Baby,Domestic Short Hair,None,Female,Calico,None,None,Small,...,1,None,UT,None,0,None,58680029,1,None,58680029
3,Dog,Mauve,Senior,Pit Bull Terrier,None,Female,Yellow / Tan / Blond / Fawn,White / Cream,None,Medium,...,1,None,UT,None,0,Mauve&amp;#39;s a gentle soul who wants nothin...,58678744,2,None,58678744
4,Cat,Howl,Baby,Domestic Short Hair,None,Male,Black,None,None,Small,...,1,None,UT,None,0,None,58670582,1,None,58670582
5,Dog,Boogie,Senior,Pit Bull Terrier,None,Male,Black,White / Cream,None,Large,...,1,None,UT,None,0,"Meet Boogie- a sweet, good looking, distinguis...",58670581,4,None,58670581
6,Dog,Mahogany,Young,Australian Shepherd,None,Male,Gray / Blue / Silver,White / Cream,None,Medium,...,1,None,UT,None,0,Meet Mahogany!\nHe&amp;#39;s a huge puppy with...,58656740,4,None,58656740
7,Dog,Sadie,Senior,Australian Shepherd,Border Collie,Female,Brown / Chocolate,White / Cream,None,Medium,...,1,None,UT,None,0,"Sadie is a sweet, senior girl who enjoys walks...",58656739,2,None,58656739
8,Cat,Totoro,Baby,Domestic Short Hair,None,Male,Black,White,None,Small,...,1,None,UT,None,0,None,58655401,2,None,58655401
9,Cat,Tucker,Adult,Domestic Short Hair,None,Male,Gray / Blue / Silver,White,None,Small,...,1,None,UT,None,0,"Tucker is a youthful, playful cat who wants no...",58655400,3,None,58655400


In [10]:
#df.to_csv('data/best_friends_animal_society/test.csv', index = False)

In [11]:
# Applying harmonization logic.
from petfinder_animal import petfinder_animal
fourth_animal = json_response['animals'][4]

harmonized_animal = petfinder_animal.create_harmonized_petfinder_animal(fourth_animal)

# Converting harmonized animal to dataframe.
print(harmonized_animal.as_kaggle_dictionary())
harmonized_animal_df = pd.DataFrame([harmonized_animal.as_kaggle_dictionary()])
harmonized_animal_df

{'Type': 2, 'Breed1': 0, 'Breed2': 0, 'Gender': 1, 'Color1': 1, 'Color2': 0, 'MaturitySize': 1, 'FurLength': 0, 'Vaccinated': 1, 'Dewormed': 3, 'Sterilized': 1, 'Health': 0, 'State': 0, 'HasDescription': False, 'Age': 0, 'Quantity': 1, 'Fee': 0, 'VideoAmt': 0, 'PhotoAmt': 1, 'DescriptionLength': 0, 'nltk_negative_prob': 0, 'nltk_neutral_prob': 0, 'nltk_positive_prob': 0, 'nltk_compound_score': 0}


,Type,Breed1,Breed2,Gender,Color1,Color2,MaturitySize,FurLength,Vaccinated,Dewormed,...,Age,Quantity,Fee,VideoAmt,PhotoAmt,DescriptionLength,nltk_negative_prob,nltk_neutral_prob,nltk_positive_prob,nltk_compound_score
0,2,0,0,1,1,0,1,0,1,3,...,0,1,0,0,1,0,0,0,0,0


In [12]:
# Load Model with pickle.
model = pickle.load(open('models/random_forest_classifier_kaggle.pkl','rb'))

In [13]:
# Call Prediction on dataframe.
print(model.predict(harmonized_animal_df))

[2]
